<a href="https://colab.research.google.com/github/JRasmusBm/chatbot-epsilon/blob/master/Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is the file in which we perform training of the NN

# Load Data

## In Colab

In [1]:
#from google.colab import files
#uploaded = files.upload()
#file_name = "amazon_cells_labelled.txt"
#uploaded[file_name].decode("utf-8")

## Locally

In [2]:
file_name = "../../data/amazon_cells_labelled.txt"

# Import code (from TA)

In [36]:
from pprint import pprint

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from matplotlib import pyplot
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/rasmus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/rasmus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
def preprocess_pandas(data, columns):
    df_ = pd.DataFrame(columns=columns)
    data['Sentence'] = data['Sentence'].str.lower()
    data['Sentence'] = data['Sentence'].replace('[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+', '', regex=True)                      # remove emails
    data['Sentence'] = data['Sentence'].replace('((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '', regex=True)    # remove IP address
    data['Sentence'] = data['Sentence'].str.replace('[^\w\s]','')                                                       # remove special characters
    data['Sentence'] = data['Sentence'].replace('\d', '', regex=True)                                                   # remove numbers
    for index, row in data.iterrows():
        word_tokens = word_tokenize(row['Sentence'])
        filtered_sent = [w for w in word_tokens if not w in stopwords.words('english')]
        df_ = df_.append({
            "index": row['index'],
            "Class": row['Class'],
            "Sentence": " ".join(filtered_sent[0:])
        }, ignore_index=True)
    return data

# If this is the primary file that is executed (ie not an import of another file)
if __name__ == "__main__":
    # get data, pre-process and split
    data = pd.read_csv(file_name, delimiter='\t', header=None)
    data.columns = ['Sentence', 'Class']
    data['index'] = data.index                                          # add new column index
    columns = ['index', 'Class', 'Sentence']
    data = preprocess_pandas(data, columns)                             # pre-process
    training_data, validation_data, training_labels, validation_labels = train_test_split( # split the data into training, validation, and test splits
        data['Sentence'].values.astype('U'),
        data['Class'].values.astype('int32'),
        test_size=0.10,
        random_state=0,
        shuffle=True
    )

    # vectorize data using TFIDF and transform for PyTorch for scalability
    word_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,2), max_features=50000, max_df=0.5, use_idf=True, norm='l2')
    training_data = word_vectorizer.fit_transform(training_data)        # transform texts to sparse matrix
    training_data = training_data.todense()                             # convert to dense matrix for Pytorch
    vocab_size = len(word_vectorizer.vocabulary_)
    validation_data = word_vectorizer.transform(validation_data)
    validation_data = validation_data.todense()
    train_x_tensor = torch.from_numpy(np.array(training_data)).type(torch.FloatTensor)
    train_y_tensor = torch.from_numpy(np.array(training_labels)).long()
    validation_x_tensor = torch.from_numpy(np.array(validation_data)).type(torch.FloatTensor)
    validation_y_tensor = torch.from_numpy(np.array(validation_labels)).long()

In [5]:
data.head()

,Sentence,Class,index
0,so there is no way for me to plug it in here i...,0,0
1,good case excellent value,1,1
2,great for the jawbone,1,2
3,tied to charger for conversations lasting more...,0,3
4,the mic is great,1,4


In [52]:
words = [ 
    word 
    for line in data["Sentence"] 
    for word in line.split(" ") if word != ""
]
word_counts = {}
for line in data["Sentence"]:
    for word in line.split(" ") :
        if word == "":
            continue
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1
    
unique_words = set(words)
print(f"Sentences: {len(data)}")
print(f"Words: {len(words)}")
print(f"Unique Words: {len(unique_words)}")
print(sorted(word_counts.items(), key=lambda item: -item[1]))

Sentences: 1000
Words: 10118
Unique Words: 1856
[('the', 513), ('i', 316), ('and', 310), ('it', 281), ('is', 243), ('a', 218), ('this', 206), ('to', 196), ('phone', 162), ('my', 143), ('for', 121), ('of', 120), ('not', 117), ('with', 112), ('very', 103), ('great', 97), ('was', 90), ('on', 89), ('in', 88), ('that', 80), ('good', 77), ('have', 73), ('you', 68), ('product', 55), ('quality', 49), ('had', 48), ('headset', 47), ('works', 47), ('but', 46), ('battery', 45), ('as', 45), ('its', 43), ('so', 42), ('are', 42), ('sound', 41), ('all', 41), ('use', 41), ('one', 40), ('well', 38), ('ear', 35), ('has', 34), ('would', 34), ('work', 34), ('from', 33), ('your', 32), ('dont', 31), ('like', 30), ('case', 29), ('if', 29), ('me', 28), ('than', 28), ('be', 28), ('ive', 28), ('excellent', 27), ('time', 27), ('after', 27), ('price', 27), ('no', 26), ('up', 26), ('recommend', 26), ('does', 26), ('really', 26), ('at', 24), ('im', 24), ('or', 23), ('best', 23), ('service', 23), ('get', 22), ('when'

In [45]:
print(len(training_data))
print(len(validation_data))
pprint(dir(word_vectorizer))
pprint(word_vectorizer.stop_words_)

900
100
['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_char_ngrams',
 '_char_wb_ngrams',
 '_check_params',
 '_check_stop_words_consistency',
 '_check_vocabulary',
 '_count_vocab',
 '_get_param_names',
 '_get_tags',
 '_limit_features',
 '_more_tags',
 '_sort_features',
 '_stop_words_id',
 '_tfidf',
 '_validate_custom_analyzer',
 '_validate_params',
 '_validate_vocabulary',
 '_white_spaces',
 '_word_ngrams',
 'analyzer',
 'binary',
 'build_analyzer',
 'build_preprocessor',
 'build_tokenizer',
 'decode',
 'decode_error',
 'dtype',
 'encoding',
 'fit',
 'fit_transform',
 'fixed_vocabulary_',
 'get_feature_names',
 'get_params',
 'get_st

# Build RNN

In [53]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        #text = [sent len, batch size]
        embedded = self.embedding(text)
        #embedded = [sent len, batch size, emb dim]
        output, hidden = self.rnn(embedded)
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [55]:
model = RNN(input_dim=vocab_size, embedding_dim=100, hidden_dim=256, output_dim=1)

In [60]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"There are {count_parameters(model):,} trainable parameters in the model")

There are 822,405 trainable parameters in the model


In [62]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

In [63]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc